In [ ]:
# Converting the date, time and categorical cloumns into usable formats.
import pandas as pd

train_data = pd.read_excel("/content/Data_Train.xlsx")
test_data = pd.read_excel("/content/Test_set.xlsx")

train_data.head(), train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10683 entries, 0 to 10682
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Airline          10683 non-null  object
 1   Date_of_Journey  10683 non-null  object
 2   Source           10683 non-null  object
 3   Destination      10683 non-null  object
 4   Route            10682 non-null  object
 5   Dep_Time         10683 non-null  object
 6   Arrival_Time     10683 non-null  object
 7   Duration         10683 non-null  object
 8   Total_Stops      10682 non-null  object
 9   Additional_Info  10683 non-null  object
 10  Price            10683 non-null  int64 
dtypes: int64(1), object(10)
memory usage: 918.2+ KB


(       Airline Date_of_Journey    Source Destination                  Route  \
 0       IndiGo      24/03/2019  Banglore   New Delhi              BLR → DEL   
 1    Air India       1/05/2019   Kolkata    Banglore  CCU → IXR → BBI → BLR   
 2  Jet Airways       9/06/2019     Delhi      Cochin  DEL → LKO → BOM → COK   
 3       IndiGo      12/05/2019   Kolkata    Banglore        CCU → NAG → BLR   
 4       IndiGo      01/03/2019  Banglore   New Delhi        BLR → NAG → DEL   
 
   Dep_Time  Arrival_Time Duration Total_Stops Additional_Info  Price  
 0    22:20  01:10 22 Mar   2h 50m    non-stop         No info   3897  
 1    05:50         13:15   7h 25m     2 stops         No info   7662  
 2    09:25  04:25 10 Jun      19h     2 stops         No info  13882  
 3    18:05         23:30   5h 25m      1 stop         No info   6218  
 4    16:50         21:35   4h 45m      1 stop         No info  13302  ,
 None)

In [ ]:
# Handeling missing values(NaNs).
train_data['Journey_Day'] = pd.to_datetime(train_data['Date_of_Journey'], format='%d/%m/%Y').dt.day
train_data['Journey_Month'] = pd.to_datetime(train_data['Date_of_Journey'], format='%d/%m/%Y').dt.month

train_data['Dep_Hour'] = pd.to_datetime(train_data['Dep_Time']).dt.hour
train_data['Dep_Min'] = pd.to_datetime(train_data['Dep_Time']).dt.minute

train_data['Arrival_Hour'] = pd.to_datetime(train_data['Arrival_Time']).dt.hour
train_data['Arrival_Minute'] = pd.to_datetime(train_data['Arrival_Time']).dt.minute

def convert_duration(duration):
  parts = duration.split()
  hours = int(parts[0].replace('h','')) if 'h' in parts[0] else 0
  minutes = int(parts[1].replace('m','')) if len(parts) > 1 else 0
  return hours * 60 + minutes

  train_data['Duration'] = train_data['Duration'].apply(convert_duration)
  train_data['Total_Stops'] = train_data['Total_Stops'].replace('non-stop', '0 stop')

  train_data_cleaned = train_data.drop(['Date_of_Journey', 'Dep_Time', 'Arrival_Time', 'Duration', 'Route'], axis=1)

  train_data_cleaned.head()

<ipython-input-25-314a9436d370>:5: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  train_data['Dep_Hour'] = pd.to_datetime(train_data['Dep_Time']).dt.hour
<ipython-input-25-314a9436d370>:6: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  train_data['Dep_Min'] = pd.to_datetime(train_data['Dep_Time']).dt.minute
<ipython-input-25-314a9436d370>:8: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  train_data['Arrival_Hour'] = pd.to_datetime(train_data['Arrival_Time']).dt.hour
<ipython-input-25-314a9436d370>:9: UserWarning: Could not infer format, so each element will be parsed individua

In [ ]:
train_data['Total_Stops'].fillna(train_data['Total_Stops'].mode()[0], inplace=True)

train_data['Total_Stops'] = train_data['Total_Stops'].replace('non-stop', '0 stop')

train_data['Total_Stops'] = train_data['Total_Stops'].str.split(' ').str[0].astype(int)

train_data_cleaned = train_data.drop(['Date_of_Journey', 'Dep_Time', 'Arrival_Time', 'Duration', 'Route'], axis=1)

train_data_cleaned.head()

,Airline,Source,Destination,Total_Stops,Additional_Info,Price,Journey_Day,Journey_Month,Dep_Hour,Dep_Min,Arrival_Hour,Arrival_Minute
0,IndiGo,Banglore,New Delhi,0,No info,3897,24,3,22,20,1,10
1,Air India,Kolkata,Banglore,2,No info,7662,1,5,5,50,13,15
2,Jet Airways,Delhi,Cochin,2,No info,13882,9,6,9,25,4,25
3,IndiGo,Kolkata,Banglore,1,No info,6218,12,5,18,5,23,30
4,IndiGo,Banglore,New Delhi,1,No info,13302,1,3,16,50,21,35


In [ ]:
missing_values = train_data_cleaned.isnull().sum()
print(missing_values[missing_values > 0])

Series([], dtype: int64)


In [ ]:
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, r2_score, mean_squared_error
import pickle

# Encode categorial variables
label_encoder = LabelEncoder()
train_data_cleaned['Airline'] = label_encoder.fit_transform(train_data_cleaned['Airline'])
train_data_cleaned['Source'] = label_encoder.fit_transform(train_data_cleaned['Source'])
train_data_cleaned['Destination'] = label_encoder.fit_transform(train_data_cleaned['Destination'])
train_data_cleaned['Additional_Info'] = label_encoder.fit_transform(train_data_cleaned['Additional_Info'])

# Split the data into features and target
X = train_data_cleaned.drop('Price', axis=1)
y = train_data_cleaned['Price']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# RandomForestREgressor model
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Predictions
y_pred = model.predict(X_test)

# accuracy = accuracy_score(y_test, y_pred)
# print(f"Accuracy: {accuracy * 100:.2f}%")

# Evaluation
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = mean_squared_error(y_test, y_pred, squared=False)

print(f"Mean Absolute Error: {mae}")
print(f"R-squared: {r2}")
print(f"Mean Squared Error: {mse}")
print(f"Root Mean Squared Error: {rmse}")

with open('flight_price_model.pkl', 'wb') as file:
    pickle.dump(model, file)

Mean Absolute Error: 634.7123188590903
R-squared: 0.9088330342834564
Mean Squared Error: 1929454.5678242776
Root Mean Squared Error: 1389.0480797381629
